In [47]:
!pip install -q transformers datasets scikit-learn accelerate


In [48]:
import numpy as np
print(f"NumPy version: {np.__version__}")  # Should show 1.26.4

NumPy version: 2.0.2


In [49]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import load_dataset, Dataset
from sklearn.metrics import accuracy_score
import torch
import numpy as np
import pandas as pd

In [50]:
import pandas as pd
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    set_seed
)
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


In [51]:
model_name = "l3cube-pune/marathi-bert"
# model_name = "aryanx16/mahabert-v2-daft"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)  # 3 labels: pos/neg/neutral


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at l3cube-pune/marathi-bert and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [52]:
# from google.colab import drive
# drive.mount('/content/drive')

In [53]:
# train_df = pd.read_csv("/content/drive/MyDrive/L3Cube/datasets/MarathiNLP-main/L3Cube-MahaSent-MD/MahaSent_ST/MahaSent_ST_Train.csv")
# val_df = pd.read_csv("/content/drive/MyDrive/L3Cube/datasets/MarathiNLP-main/L3Cube-MahaSent-MD/MahaSent_ST/MahaSent_ST_Val.csv")
# test_df = pd.read_csv('/content/drive/MyDrive/L3Cube/datasets/MarathiNLP-main/L3Cube-MahaSent-MD/MahaSent_ST/MahaSent_ST_Test.csv')
# train_df.head()
# val_df.head()
# test_df.head()
train_df = pd.read_csv("/content/drive/MyDrive/L3Cube/datasets/MarathiNLP-main/L3Cube-MahaSent-MD/MahaSent_MR/MahaSent_MR_Train.csv")
val_df = pd.read_csv("/content/drive/MyDrive/L3Cube/datasets/MarathiNLP-main/L3Cube-MahaSent-MD/MahaSent_MR/MahaSent_MR_Val.csv")
test_df = pd.read_csv('/content/drive/MyDrive/L3Cube/datasets/MarathiNLP-main/L3Cube-MahaSent-MD/MahaSent_MR/MahaSent_MR_Test.csv')
train_df.head()
# val_df.head()
# test_df.head()

,Unnamed: 0,marathi_sentence,label
0,0,माने यांचा घटस्फोट झाला आहे तर मोहितेने नुकतेच...,-1
1,1,एका रात्रीत घडणारी किंबहुना बिघडणारी ही गोष्ट आहे,-1
2,2,जरी आघात समजण्यायोग्य आहे जरी चित्रपटाला खराब ...,-1
3,3,पण तो असा आघातही अनुभवत आहे की तो कोणाशीही शेअ...,-1
4,4,छोटे-छोटे गैरसमज मोठ्या अडचणीत येतात,-1


In [54]:
# train_df["label"] += 1
# val_df["label"] += 1
# test_df["label"] += 1
# train_df.head()
# train_df.dropna(subset=["marathi_text", "label"], inplace=True)
# val_df.dropna(subset=["marathi_text", "label"], inplace=True)
# test_df.dropna(subset=["marathi_text", "label"], inplace=True)
# print(train_df.isnull().sum())





train_df["label"] += 1
val_df["label"] += 1
test_df["label"] += 1
train_df.head()
train_df.dropna(subset=["marathi_sentence", "label"], inplace=True)
val_df.dropna(subset=["marathi_sentence", "label"], inplace=True)
test_df.dropna(subset=["marathi_sentence", "label"], inplace=True)
print(train_df.isnull().sum())

Unnamed: 0          0
marathi_sentence    0
label               0
dtype: int64


In [55]:
# Step 3: Convert to Hugging Face Dataset format
from datasets import Dataset

train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

In [56]:
#  Step 4: Tokenize the tweets
def tokenize_function(batch):
    return tokenizer(batch["marathi_sentence"], padding="max_length", truncation=True, max_length=128)

train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/12000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [57]:
# Format for PyTorch
columns = ['input_ids', 'attention_mask', 'label']
train_dataset.set_format(type='torch', columns=columns)
val_dataset.set_format(type='torch', columns=columns)
test_dataset.set_format(type='torch', columns=columns)

In [58]:

#  Initialize Trainer

training_args = TrainingArguments(
    output_dir="./results",                   # 📁 Where to save checkpoints
    eval_strategy="epoch",              # 🧪 Evaluate after each epoch
    save_strategy="epoch",                    # 💾 Save model after each epoch
    per_device_train_batch_size=64,            # 🧠 Reduce memory usage
    per_device_eval_batch_size=64,             # ⚖️ Matching batch size for eval
    num_train_epochs=2,                       # 🔁 You can try 1 or 2 for speed
    learning_rate=2e-5,                       # 🚀 Good default for BERT
    # warmup_steps=100,                         # 🔥 Warmup helps training stability
    weight_decay=0.01,                        # 📉 Helps avoid overfitting
    logging_dir="./logs",                     # 📂 Logs directory
    logging_steps=50,                         # 📝 Log every 50 steps
    load_best_model_at_end=True,              # 🏆 Automatically load best model
    metric_for_best_model="accuracy",         # 🎯 Use accuracy to pick best
    save_total_limit=1,                       # 💾 Keep only last best model
    fp16=True,                                # ⚡ Mixed precision for faster training
    report_to="none",                         # 📊 Turn off W&B if you’re not using it
    seed=42,         # Standard seed, might be what they used
data_seed=42,
)
# training_args = TrainingArguments(
#     learning_rate=2e-5,
#     num_train_epochs=2,
#     per_device_train_batch_size=64,
#     per_device_eval_batch_size=64,
#     weight_decay=0.01,
#     warmup_steps=0,
#     eval_strategy="epoch",
#     save_strategy="epoch",
#     load_best_model_at_end=True,
#     fp16=True,
#     report_to="none"
# )



In [59]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {'accuracy': accuracy_score(labels, predictions)}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

/tmp/ipython-input-485065071.py:6: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [60]:
# Step 5: Train!
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.650800,0.634865,0.732667
2,0.505300,0.630032,0.744667


TrainOutput(global_step=376, training_loss=0.611843449004153, metrics={'train_runtime': 202.9719, 'train_samples_per_second': 118.243, 'train_steps_per_second': 1.852, 'total_flos': 1578680506368000.0, 'train_loss': 0.611843449004153, 'epoch': 2.0})

In [61]:
# After training
val_metrics = trainer.evaluate()          # still okay to see validation
print("Validation metrics:", val_metrics)

test_metrics = trainer.evaluate(test_dataset)   # ✅ this matches the paper
print("Test metrics:", test_metrics)


Validation metrics: {'eval_loss': 0.6300320625305176, 'eval_accuracy': 0.7446666666666667, 'eval_runtime': 2.411, 'eval_samples_per_second': 622.147, 'eval_steps_per_second': 9.954, 'epoch': 2.0}
Test metrics: {'eval_loss': 0.5906155705451965, 'eval_accuracy': 0.7613333333333333, 'eval_runtime': 2.383, 'eval_samples_per_second': 629.45, 'eval_steps_per_second': 10.071, 'epoch': 2.0}
